In [1]:
using PauliPropagation

In [2]:
nq = 8

8

In [3]:
op = PauliString(nq, :Z, round(Int, nq/2))

PauliString(nqubits: 8, 1.0 * IIIZIIII)

In [4]:
nl = 4
topo = bricklayertopology(nq; periodic=false)
circ = hardwareefficientcircuit(nq, nl; topology=topo)
m = length(circ)

124

In [5]:
using Random
Random.seed!(42)
thetas = randn(m);

In [6]:
W = Inf;                   # maximal operator weight.
min_abs_coeff = 0;          # neglect small coefficients. Only for numerical and hybrid PP.

#### Numerical Pauli Propagation
Propagates numerical coefficients.

In [7]:
opsum = PauliSum(nq, op)

PauliSum(nqubits: 8, 1 Pauli term: 
 1.0 * IIIZIIII
)

In [8]:
@time dnum = mergingbfs(circ, op, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.601890 seconds (754.52 k allocations: 53.776 MiB, 1.85% gc time, 82.95% compilation time)
length(dnum) = 53247


0.21720058439757214

In [9]:
@time dnum = mergingbfs(circ, opsum, thetas; max_weight=W, min_abs_coeff=min_abs_coeff);
@show length(dnum)   # number of unique Pauli ops
overlapwithzero(dnum) # expectation

  0.107985 seconds (414 allocations: 3.790 MiB, 5.92% compilation time)
length(dnum) = 53247


0.21720058439757214

In [10]:
dnum

PauliSum(nqubits: 8, 53247 Pauli terms:
 9.3421e-7 * XZIIXXZX
 9.3651e-6 * IIIXIIZY
 3.8189e-6 * YZZIYXZX
 -0.00014415 * ZXYYYXII
 -1.2334e-6 * IZYZIIXY
 2.2079e-7 * XXIIYZXI
 1.4947e-7 * IZZXIZYZ
 3.2705e-7 * YYXIYYYZ
 -0.0035534 * YZIXXZII
 -5.7279e-7 * IXIYIIZX
 -3.3665e-6 * IYZZZYYX
 -8.1232e-9 * YIYZYIZZ
 4.2359e-8 * XIZZXXZZ
 -2.3432e-8 * XZZXZZYZ
 -0.00012064 * ZZXZZXII
 -4.8324e-8 * XZYXIYYZ
 -3.3641e-7 * XXXYIYXZ
 9.2096e-7 * IIIXZZXZ
 5.996e-7 * ZZYXYYYY
 -2.2674e-7 * XXXXZZXY
  ⋮)

#### Hybrid Pauli Propagation
Propagates numerical coefficients, but can but truncated like the surrogate.

In [11]:
max_freq = Inf   # max frequency, i.e., max number of sines and cosines per path

Inf

In [12]:
wrapped_op = wrapcoefficients(op, NumericPathProperties)
@time dhyb = mergingbfs(circ, wrapped_op, thetas; max_weight=W, max_freq=Inf, min_abs_coeff=min_abs_coeff);
@show length(dhyb)
overlapwithzero(dhyb)

  0.432451 seconds (1.06 M allocations: 59.755 MiB, 1.90% gc time, 64.77% compilation time)
length(dhyb) = 53247


0.21720058439757214

In [13]:
dhyb

PauliSum(nqubits: 8, 53247 Pauli terms:
 PathProperty(9.3421e-7) * XZIIXXZX
 PathProperty(9.3651e-6) * IIIXIIZY
 PathProperty(3.8189e-6) * YZZIYXZX
 PathProperty(-0.00014415) * ZXYYYXII
 PathProperty(-1.2334e-6) * IZYZIIXY
 PathProperty(2.2079e-7) * XXIIYZXI
 PathProperty(1.4947e-7) * IZZXIZYZ
 PathProperty(3.2705e-7) * YYXIYYYZ
 PathProperty(-0.0035534) * YZIXXZII
 PathProperty(-5.7279e-7) * IXIYIIZX
 PathProperty(-3.3665e-6) * IYZZZYYX
 PathProperty(-8.1232e-9) * YIYZYIZZ
 PathProperty(4.2359e-8) * XIZZXXZZ
 PathProperty(-2.3432e-8) * XZZXZZYZ
 PathProperty(-0.00012064) * ZZXZZXII
 PathProperty(-4.8324e-8) * XZYXIYYZ
 PathProperty(-3.3641e-7) * XXXYIYXZ
 PathProperty(9.2096e-7) * IIIXZZXZ
 PathProperty(5.996e-7) * ZZYXYYYY
 PathProperty(-2.2674e-7) * XXXXZZXY
  ⋮)

#### Pauli Propagation Surrogate
Builds a graph that can later be evaluated.
This all looks very messy - and it is! So don't worry about it.

In [14]:
wrapped_op2 = wrapcoefficients(op, NodePathProperties)
@time dsym = mergingbfs(circ, wrapped_op2, zeros(m); max_weight=W, max_freq=max_freq);
@show length(dsym)

final_nodes = collect(pth.coeff for (obs, pth) in zerofilter(dsym));
final_eval_node = PauliGateNode(parents=final_nodes, trig_inds=zeros(Int, length(final_nodes)), signs=ones(length(final_nodes)), param_idx=1, cummulative_value=0.0);
resetnodes(final_eval_node)
resetnodes(final_eval_node)
@time eval_list = gettraceevalorder(final_eval_node, zeros(m));
length(eval_list)  # The list of all nodes. The order is such that one can savely be evaluated after the other.

  1.437624 seconds (13.05 M allocations: 716.452 MiB, 16.96% gc time, 31.65% compilation time)
length(dsym) = 53247
  0.096280 seconds (29.81 k allocations: 4.959 MiB, 42.46% compilation time)


225500

In [15]:
@time expectation(eval_list, thetas)    # This is actually not always faster than numerical propagation, but in interesting cases it is by a lot.
                                        # making this always at least as fast is work in progress. Graph traversal is hard.

  0.097848 seconds (55.70 k allocations: 3.867 MiB, 52.68% compilation time)


0.2172005843975723